In [587]:
def R2compute(data, target, var):
    #data: in form of a pandas.DataFrame
    #target: the colulmn name for the target dependant vaeriable
    #var: the column name for the independant variable
    meanP = data[target].mean()
    grouped = data.groupby(data[var])[target]
    SSM = sum(grouped.size()*(grouped.mean() - meanP)**2)
    SSR = sum(grouped.apply(lambda x: (x-x.mean())**2))
    return(SSM/(SSM+SSR))

In [589]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import skew
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import cross_val_score

train = pd.read_csv('train.csv', header = 0)
test = pd.read_csv('test.csv', header = 0)
train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)

target = 'SalePrice'

categorial = train.select_dtypes(include=['object']).columns
numerical = data.select_dtypes(include=['int64','float64']).columns
#Replace missing values
train[numerical] = train[numerical].fillna(value=train[numerical].mean())
train[categorial] = train[categorial].fillna(value='None')

test[numerical] = test[numerical].fillna(value=test[numerical].mean())
test[categorial] = test[categorial].fillna(value='None')

corrmat = train.corr()
toKeep = list(corrmat[target][abs(corrmat[target].values) >= 0].drop(target).index)
R2list = train.apply(lambda x: R2compute(train, target, x.name) if (x.name in categorial) else 0)
toKeep += list(R2list[R2list>=0.0001].index)

X = train[toKeep]
test = test[toKeep]
y = train[target]
len_X = X.shape[0]

data = pd.concat([X,test], sort=False)

#Find categorial and numerical values

#Find skewed features and apply log transformation
skewed_feats = data[numerical].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
data[skewed_feats] = np.log1p(data[skewed_feats])
#Apply log-transformation to the target
y = np.log1p(y)

#Get dummies
data = pd.get_dummies(data)

#Retrieve train and test
X = data[:len_X]
test = data[len_X:]

In [590]:
scaler = RobustScaler()
X = scaler.fit_transform(X)
test = scaler.transform(test)
model = LassoCV(alphas = [1, 0.1, 0.001, 0.0005], max_iter=10000, cv=5).fit(X, y)
print('Mean mse score : ', np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv = 5)).mean())
print('alpha chosen : ', model.alpha_)

Mean mse score :  0.12263031248711356
alpha chosen :  0.0005


In [591]:
res = np.expm1(model.predict(test))

pd.DataFrame(res, index=list(range(1461,2920)), columns=['SalePrice']).to_csv("mypred2.csv", index_label='Id')